# LLAMA-Factory微调模型

In [1]:
!pip install deepspeed

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... one
  Created wheel for deepspeed: filename=deepspeed-0.16.5-py3-none-any.whl size=1580682 sha256=ed265b736e5fdaced9356ce3ae0b275889b4ff45df40bb0398478f1d24fe2f27
  Stored in directory: /home/zhangsh82/.cache/pip/wheels/15/b3/d2/9123c71066491c377366a0fb72143f432e86d370ce08634663
Successfully built deepspeed


In [2]:
!pip install flash-attn --no-build-isolation

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.6 MB/s eta 0:00:00a 0:00:01m
  Preparing metadata (setup.py) ... one
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp310-cp310-linux_x86_64.whl size=187797312 sha256=b267f80a08e516292cdd748056a2178a45b8abedf7fca123292eb17c21c8c87c
  Stored in directory: /home/zhangsh82/.cache/pip/wheels/34/f4/ae/aebe0077ed247cabb5a74b211ffbbdb3f84147016aa2cae4ac
Successfully built flash-attn


In [2]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git

正克隆到 'LLaMA-Factory'...
remote: Enumerating objects: 353, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 353 (delta 94), reused 183 (delta 57), pack-reused 0 (from 0)
接收对象中: 100% (353/353), 9.70 MiB | 9.97 MiB/s, 完成.
处理 delta 中: 100% (94/94), 完成.


In [4]:
!pip install -e "/home/zhangsh82/data/zsh/LLaMA-Factory"

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple
Obtaining file:///home/zhangsh82/data/zsh/LLaMA-Factory
  Installing build dependencies ... one
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached https://mirrors.cloud.tencent.com/pypi/packages/73/e0/f1fce1b2f31da4acfb858732d909209fdb10f65adda1e7af0c7ac74927a5/transformers-4.51.1-py3-none-any.whl (10.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 1.6 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 6.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━

In [5]:
!llamafactory-cli version

[2025-04-11 10:16:51,975] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
----------------------------------------------------------
| Welcome to LLaMA Factory, version 0.9.3.dev0           |
|                                                        |
| Project page: https://github.com/hiyouga/LLaMA-Factory |
----------------------------------------------------------


In [7]:
!llamafactory-cli --help

[2025-04-11 10:17:48,121] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Unknown command: --help.
----------------------------------------------------------------------
| Usage:                                                             |
|   llamafactory-cli api -h: launch an OpenAI-style API server       |
|   llamafactory-cli chat -h: launch a chat interface in CLI         |
|   llamafactory-cli eval -h: evaluate models                        |
|   llamafactory-cli export -h: merge LoRA adapters and export model |
|   llamafactory-cli train -h: train models                          |
|   llamafactory-cli webchat -h: launch a chat interface in Web UI   |
|   llamafactory-cli webui: launch LlamaBoard                        |
|   llamafactory-cli version: show version info                      |
----------------------------------------------------------------------


In [2]:
!pwd

/home/zhangsh82/data/zsh


In [31]:
%%bash

#!/bin/bash
export CUDA_VISIBLE_DEVICES=7

# 设置节点数为1
export NNODES=1
# 设置每个节点上的GPU数量为1
export GPUS_PER_NODE=1
# 设置当前节点的等级为0
export NODE_RANK=0
# 设置主节点的地址为本地IP
export MASTER_ADDR=localhost
# 设置主节点的端口为1234
export MASTER_PORT=1234

MODEL='/home/zhangsh82/.cache/modelscope/hub/models/Qwen/Qwen2.5-0.5B-Instruct/'
OUTPUT_PATH='/home/zhangsh82/.cache/modelscope/hub/models/Qwen/Qwen2.5-0.5B-zhenhuan'
DATA_SET='zhenhuan'
DS_CONFIG_PATH='/home/zhangsh82/data/zsh/LLaMA-Factory/examples/deepspeed/ds_z3_config.json'


DISTRIBUTED_ARGS="
    --nproc_per_node $GPUS_PER_NODE \
    --nnodes $NNODES \
    --node_rank $NODE_RANK \
    --master_addr $MASTER_ADDR \
    --master_port $MASTER_PORT
"

torchrun $DISTRIBUTED_ARGS LLaMA-Factory/src/train.py \
    --deepspeed $DS_CONFIG_PATH \
    --stage sft \
    --do_train \
    --use_fast_tokenizer \
    --flash_attn disabled \
    --model_name_or_path $MODEL \
    --dataset $DATA_SET \
    --template qwen \
    --finetuning_type lora \
    --lora_target q_proj,v_proj\
    --output_dir $OUTPUT_PATH \
    --overwrite_cache \
    --overwrite_output_dir \
    --warmup_steps 100 \
    --weight_decay 0.1 \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --ddp_timeout 9000 \
    --learning_rate 5e-6 \
    --lr_scheduler_type cosine \
    --logging_steps 1 \
    --cutoff_len 4096 \
    --save_steps 1000 \
    --plot_loss \
    --num_train_epochs 3 \
    --fp16

[2025-04-11 15:02:06,127] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-04-11 15:02:08,242] [INFO] [comm.py:658:init_distributed] cdb=None
INFO] [comm.py:689:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[WARNING|2025-04-11 15:02:08] llamafactory.hparams.parser:148 >> `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
[INFO|2025-04-11 15:02:08] llamafactory.hparams.parser:379 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: True, compute dtype: torch.float16


[INFO|tokenization_utils_base.py:2058] 2025-04-11 15:02:08,320 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2058] 2025-04-11 15:02:08,320 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2058] 2025-04-11 15:02:08,320 >> loading file tokenizer.json
tokenization_utils_base.py:2058] 2025-04-11 15:02:08,320 >> loading file added_tokens.json
enization_utils_base.py:2058] 2025-04-11 15:02:08,320 >> loading file special_tokens_map.json
tokenization_utils_base.py:2058] 2025-04-11 15:02:08,320 >> loading file tokenizer_config.json
|tokenization_utils_base.py:2058] 2025-04-11 15:02:08,320 >> loading file chat_template.jinja
[INFO|tokenization_utils_base.py:2323] 2025-04-11 15:02:08,682 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO|configuration_utils.py:691] 2025-04-11 15:02:08,683 >> loading configuration file /home/zhangsh82/.cache/modelscope/hub/models/Qwen/Qwen2.5-0.5B-Instruct/config.j

[INFO|2025-04-11 15:02:09] llamafactory.data.template:143 >> Add <|im_end|> to stop words.


[rank0]: Traceback (most recent call last):
src/llamafactory/data/parser.py", line 107, in get_dataset_list
as f:0]:     with open(config_path) 
[rank0]: FileNotFoundError: [Errno 2] No such file or directory: 'data/dataset_info.json'

nk0]: During handling of the above exception, another exception occurred:

recent call last): (most 
n <module> File "/home/zhangsh82/data/zsh/LLaMA-Factory/src/train.py", line 28, i
[rank0]:     main()
, line 19, in mainhome/zhangsh82/data/zsh/LLaMA-Factory/src/train.py"
[rank0]:     run_exp()
c/llamafactory/train/tuner.py", line 107, in run_expory/sr
 args, "callbacks": callbacks})(config={"args":
tory/train/tuner.py", line 69, in _training_functionory/src/llamafac
ining_args, finetuning_args, generating_args, callbacks)
LaMA-Factory/src/llamafactory/train/sft/workflow.py", line 51, in run_sft
e = get_dataset(template, model_args, data_args, training_args, stage="sft", **tokenizer_module)
nk0]:   File "/home/zhangsh82/data/zsh/LLaMA-Factory/src/llamafa

CalledProcessError: Command 'b'\n#!/bin/bash\nexport CUDA_VISIBLE_DEVICES=7\n\n# \xe8\xae\xbe\xe7\xbd\xae\xe8\x8a\x82\xe7\x82\xb9\xe6\x95\xb0\xe4\xb8\xba1\nexport NNODES=1\n# \xe8\xae\xbe\xe7\xbd\xae\xe6\xaf\x8f\xe4\xb8\xaa\xe8\x8a\x82\xe7\x82\xb9\xe4\xb8\x8a\xe7\x9a\x84GPU\xe6\x95\xb0\xe9\x87\x8f\xe4\xb8\xba1\nexport GPUS_PER_NODE=1\n# \xe8\xae\xbe\xe7\xbd\xae\xe5\xbd\x93\xe5\x89\x8d\xe8\x8a\x82\xe7\x82\xb9\xe7\x9a\x84\xe7\xad\x89\xe7\xba\xa7\xe4\xb8\xba0\nexport NODE_RANK=0\n# \xe8\xae\xbe\xe7\xbd\xae\xe4\xb8\xbb\xe8\x8a\x82\xe7\x82\xb9\xe7\x9a\x84\xe5\x9c\xb0\xe5\x9d\x80\xe4\xb8\xba\xe6\x9c\xac\xe5\x9c\xb0IP\nexport MASTER_ADDR=localhost\n# \xe8\xae\xbe\xe7\xbd\xae\xe4\xb8\xbb\xe8\x8a\x82\xe7\x82\xb9\xe7\x9a\x84\xe7\xab\xaf\xe5\x8f\xa3\xe4\xb8\xba1234\nexport MASTER_PORT=1234\n\nMODEL=\'/home/zhangsh82/.cache/modelscope/hub/models/Qwen/Qwen2.5-0.5B-Instruct/\'\nOUTPUT_PATH=\'/home/zhangsh82/.cache/modelscope/hub/models/Qwen/Qwen2.5-0.5B-zhenhuan\'\nDATA_SET=\'zhenhuan\'\nDS_CONFIG_P

In [ ]:
CUDA_VISIBLE_DEVICES=7 llamafactory-cli export \
    --model_name_or_path /home/zhangsh82/.cache/modelscope/hub/models/Qwen/Qwen2.5-0.5B-Instruct/ \
    --adapter_name_or_path /home/zhangsh82/.cache/modelscope/hub/models/Qwen/Qwen2.5-0.5B-zhenhuan \
    --template qwen \
    --finetuning_type lora \
    --export_dir /home/zhangsh82/.cache/modelscope/hub/models/Qwen/Qwen2.5-0.5B-zhenhuan-lora-merge \
    --export_size 2 \
    --export_legacy_format False